# 0. 드라이브 연결

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. jsonl 데이터 로드

In [ ]:
from datasets import load_dataset

# 학습 데이터 준비
dataset = load_dataset(
    "json",
    data_files={"train": "/content/drive/MyDrive/DILAB/YB/GPT_FineTuning/news_data/*.jsonl"}   # 구글 드라이브 데이터 경로 지정
)["train"]
print(dataset)

Resolving data files:   0%|          | 0/34 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title', 'body'],
    num_rows: 1231
})


# 2. 불러온 데이터들을 Train/Valid로 분할

In [ ]:
# .train_test_split() : 하나의 Dataset을 train용과 test용으로 분리하는 함수
  # test_size = 0.1 : test 사이즈를 10, train 사이즈를 90으로 설정함
  # seed = 42 : shuffle할 시드값을 지정한 것이다. 해당 시드값을 지정해놓으면, 다시 shuffle해도 똑같은 결과값이 나온다.
splits = dataset.train_test_split(test_size=0.1, seed=42)  # 90/10
train_dataset = splits["train"]
valid_dataset = splits["test"]

# 3. train/valid 데이터들 토크나이징

In [ ]:
from transformers import AutoTokenizer

# 저장해둔 모델로부터 토크나이저 가져오기
model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 스페셜 토큰 등록, 저장하기
specials = {"additional_special_tokens" : ["<title>", "</title>", "<body>", "</body>"]}
tokenizer.add_special_tokens(specials)
tokenizer.save_pretrained(model_path)

  # to_one_batch() : 제목/본문을 하나의 입력 문자열로 합치는 함수
def to_one_batch(ex):
  return {"text" : "<title>" + ex["title"] + "</title>" + "<body>" + ex["body"] + "</body>"}


# 불러온 데이터들에 to_one_batch 함수 적용하기
  # .map() 함수는 인자로 지정해놓은 함수를 거친 결과를 기존 데이터셋에 추가하는 함수이다.
  # 따라서, "remove_columns"를 지정하지 않으면 해당 데이터셋에 있는 "title"과 "body"가 그대로 남아있으므로, remove_columns를 지정하는 것이다.
  # 이렇게 되면 해당 데이터셋에는 "text" 컬럼의 데이터만 남게 된다.
train_dataset = train_dataset.map(to_one_batch, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(to_one_batch, remove_columns=valid_dataset.column_names)

  # tokenize_function() : 인자로 들어오는 batch를 토크나이즈 하는 함수
max_length = 2048 # 생성할 토큰 최대 길이
def tokenize_function(batch):
  return tokenizer( # 사전학습된 토크나이저 객체
    batch["text"], # 변환할 원본 텍스트 (문자열 or 문자열 리스트)
    truncation=True, # 문장이 너무 길면 max_length에 맞춰 잘라낼지 여부 결정(절단)
    padding = "max_length", # 문장이 짧은 경우, max_length 길이에 맞춰서 뒤에 [PAD] 토큰 붙여서 길이 맞추기
    max_length=max_length, # 최대 토큰 길이 설정
    return_tensors = "pt" # tokenizer는 파이썬의 dict 형태로 결과를 주는데, 해당 return을 Pytorch Tensor 형태로 반환하도록 설정
  )


train_tokenized = train_dataset.map(tokenize_function, batched=True) # 한번에 여러 샘플 묶음(배치)을 리스트/딕셔너리 형태로 전달하기 위한 설정
valid_tokenized = valid_dataset.map(tokenize_function, batched=True)

print(train_tokenized['text'])



Map:   0%|          | 0/752 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/752 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Column(['<title>강원광역 여성새로일하기 센터, 강원 여성IT전문강사 양성과정 교육생 모집강원도민일보1일 전네이버뉴스</title><body>강원광역 여성새로일하기 센터가 14일까지 강원 여성 IT전문강사 양성과정에 참여할 교육생을 모집한다.\n강원 여성 인공지능 역량 향상과 관련분야 진출을 돕기 위한 기획된 것으로 강원광역새일센터와 KT가 올해 처음 공동운영한다.\n강원도 경력단절여성을 대상으로 모집하며 총 74명을 선정해 춘천, 원주, 강릉, 화천 등 4개 지역에서 교육을 진행한다.\n자세한 사항은 강원광역 여성새로일하기 센터에 문의하면 된다. 이기영 기자\n================================================================================</body>', '<title>北노동자 "신원위장하면 美IT업체도 취업…외화벌이 85% 송금"연합뉴스7시간 전네이버뉴스</title><body>탈북 남성 BBC 인터뷰…中→헝가리·튀르키예→유럽·미국 \'다단계\' 신분세탁\n임화섭 기자 = 북한에서 중국으로 파견된 후 \'다단계 위장 신원\'으로 미국과 유럽의 기업들에 취업해 원격근무로 외화벌이를 했던 북한 출신 정보기술 노동자의 사연을 영국 BBC 방송이 2일 전했다.\nBBC는 이 인물을 영상통화로 직접 인터뷰했으며 신원 보호를 위해 기사에는 \'진수\'라는 가명을 썼다고 설명했다.\n진수는 중국에서 일하면서 원격근무로 미국과 유럽 기업들로부터 일감을 따내 매월 최소한 5천 달러을 벌어왔다며 함께 일한 동료 중에는 훨씬 더 많은 벌어온 경우도 있다고 말했다.\n진수는 그와 동료들이 보통 10명씩 팀을 이뤄 일했다고 말했다.\n진수는 벌어온 돈의 85%가 북한으로 갔다며 "강도행위 같다는 것을 우리도 알지만 그냥 운명으로 받아들인다"며 "그래도 북한에 있을 때보다는 훨씬 낫다"고 말했다.\n진수처럼 중국과 러시아, 혹은 아프리카나 다른 해외 지역으로 파견돼 이런 방법으로 외화벌이에 동원되는 북한 I

# 4. 토크나이즈한 값들 드라이브에 저장

In [ ]:
train_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/train_tokenized")
valid_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/valid_tokenized")

NameError: name 'train_tokenized' is not defined